In [4]:
# -*- coding: utf-8 -*-
"""Generate a *DefMap* video (heat‑map of displacement speed) with the
**Lucas‑Kanade** optical‑flow algorithm – in exakt dem Stil deines
Referenz‑PNG (plasma colour‑map, Color‑Bar rechts, Titel oben).

Workflow
========
1. Lade das Timelapse‑Video.
2. Croppe einen konfigurierbaren ROI, damit nur der relevante Bildausschnitt
   ausgewertet wird.
3. Berechne **dichten** Lucas‑Kanade‑Fluss: wir legen ein gleichmässiges
   Gitternetz von Seed‑Punkten (Rasterweite *GRID_STEP*) und propagieren es
   frame‑für‑frame mit `cv2.calcOpticalFlowPyrLK`.  Die resultierenden sparse
   Vektoren werden in zwei 2‑D‑Matrizen (dx, dy) projiziert und anschliessend
   per Gauß‑Blur geglättet, um ein pseudodichtes Feld zu erhalten.
4. Leite daraus die Geschwindigkeit ‖v‖ ab.
5. Rendere jedes Frame mit Matplotlib (plasma‑Colormap; fester Wertebereich
   0 → blau bis *MAX_SPEED_VIS* → gelb) – Titel: *DefMap at Time: t*; Color‑Bar
   rechts.
6. Schreibe das Ganze als MP4 *und* speichere Basis‑Metriken pro Frame in
   einer CSV.

Dieser Ansatz ist etwas langsamer als Farnebäck, produziert aber das gewünschte
Lucas‑Kanade‑basiertes Ergebnis.  Für Sequenzen mit >1000 Frames ggf.
*DOWNSAMPLE* erhöhen.
"""
from __future__ import annotations

import cv2
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")  # head‑less backend (kein GUI nötig)
import matplotlib.pyplot as plt
from pathlib import Path

# ----------------------------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------------------------
ROOT_DIR = Path("/Users/macbook/Library/Mobile Documents/com~apple~CloudDocs/Studium/Spezialisierung/CodeExampleRayan")
INPUT_VIDEO = ROOT_DIR / "data/input/LinearStackAlignmentSift_Gauss5px.avi"
OUTPUT_DIR = ROOT_DIR / "data/output/defmap_video_lk"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
VIDEO_OUT = OUTPUT_DIR / "defmap_lk_plasma.mp4"
CSV_OUT = OUTPUT_DIR / "defmap_lk_speed_metrics.csv"

ROI_MARGIN = 100      # Rand­abstand in Pixeln, der weggeschnitten wird
GRID_STEP = 5         # Gitterabstand (Pixel) der Seed‑Punkte
SMOOTH_K = 5          # Kernelgröße für Gauß‑Blur → glättet das sparse Feld
DOWNSAMPLE = 1        # jede n‑te Frame verarbeiten (>=1)
FPS_OUT = 10          # FPS des Ausgabevideos
MAX_SPEED_VIS = 10.0  # oberes Ende der Color‑Bar (µm oder Pixel pro Frame)
FIG_DPI = 150         # Auflösung der Matplotlib‑Figures

# Lucas‑Kanade‑Parameter (bewährt für Zell‑Timelapse)
LK_ARGS = dict(
    winSize=(11, 11),
    maxLevel=3,
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01)
)
# ----------------------------------------------------------------------------

def dense_lucas_kanade(prev_g: np.ndarray, next_g: np.ndarray) -> np.ndarray:
    """Berechne ein pseudodichtes Lucas‑Kanade‑Flussfeld via Gittersampling."""
    h, w = prev_g.shape
    ys, xs = np.mgrid[0:h:GRID_STEP, 0:w:GRID_STEP]
    p0 = np.stack((xs.flatten(), ys.flatten()), axis=-1).astype(np.float32)
    p0 = p0.reshape(-1, 1, 2)

    p1, st, _ = cv2.calcOpticalFlowPyrLK(prev_g, next_g, p0, None, **LK_ARGS)
    st = st.reshape(-1)
    p0 = p0.reshape(-1, 2)
    p1 = p1.reshape(-1, 2)

    flow = np.zeros((h, w, 2), dtype=np.float32)
    flow_valid = p1[st == 1] - p0[st == 1]
    pts_valid = p0[st == 1].astype(int)

    flow[pts_valid[:, 1], pts_valid[:, 0]] = flow_valid
    # Gauß‑Blur glättet & füllt Lücken → pseudodicht
    flow[..., 0] = cv2.GaussianBlur(flow[..., 0], (SMOOTH_K, SMOOTH_K), 0)
    flow[..., 1] = cv2.GaussianBlur(flow[..., 1], (SMOOTH_K, SMOOTH_K), 0)
    return flow


def flow_speed(flow: np.ndarray) -> np.ndarray:
    dx, dy = flow[..., 0], flow[..., 1]
    return np.sqrt(dx ** 2 + dy ** 2)


def render_defmap(speed: np.ndarray, frame_no: int) -> np.ndarray:
    """Render Matplotlib‑Figure → RGB‑Array."""
    fig = plt.figure(figsize=(4, 4), dpi=FIG_DPI)
    ax = fig.add_axes([0.05, 0.05, 0.8, 0.9])
    im = ax.imshow(speed, cmap="plasma", vmin=0, vmax=MAX_SPEED_VIS)
    ax.set_axis_off()
    ax.set_title(f"DefMap at Time: {frame_no}", fontsize=14, weight="bold")

    cb_ax = fig.add_axes([0.88, 0.1, 0.03, 0.8])
    cbar = fig.colorbar(im, cax=cb_ax)
    cbar.set_ticks([0, MAX_SPEED_VIS])
    cbar.set_ticklabels(["0", f"{int(MAX_SPEED_VIS)}"])

    fig.canvas.draw()
    w_canvas, h_canvas = fig.canvas.get_width_height()
    rgb = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    plt.close(fig)
    return rgb.reshape((h_canvas, w_canvas, 3))


# ----------------------------------------------------------------------------
# MAIN PIPELINE
# ----------------------------------------------------------------------------
cap = cv2.VideoCapture(str(INPUT_VIDEO))
if not cap.isOpened():
    raise IOError(f"Cannot open {INPUT_VIDEO}")

ret, frame = cap.read()
if not ret:
    raise RuntimeError("Input video empty")

prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)[ROI_MARGIN:-ROI_MARGIN, ROI_MARGIN:-ROI_MARGIN]

# initial dummy‑figure → dimension for VideoWriter
frame0_rgb = render_defmap(np.zeros_like(prev_gray, dtype=float), 0)
H_fig, W_fig, _ = frame0_rgb.shape
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(str(VIDEO_OUT), fourcc, FPS_OUT, (W_fig, H_fig))

metrics = []
frame_idx = 0

while True:
    # evtl. Frames überspringen
    for _ in range(DOWNSAMPLE):
        ret, frame = cap.read()
        if not ret:
            break
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)[ROI_MARGIN:-ROI_MARGIN, ROI_MARGIN:-ROI_MARGIN]
    flow = dense_lucas_kanade(prev_gray, gray)
    speed = flow_speed(flow)

    rgb_fig = render_defmap(speed, frame_idx)
    writer.write(cv2.cvtColor(rgb_fig, cv2.COLOR_RGB2BGR))

    metrics.append({
        "frame": frame_idx,
        "speed_mean": float(np.mean(speed)),
        "speed_std": float(np.std(speed)),
        "speed_max": float(np.max(speed)),
    })

    prev_gray = gray
    frame_idx += 1

cap.release()
writer.release()

pd.DataFrame(metrics).to_csv(CSV_OUT, index=False)
print(f"✅ DefMap (Lucas‑Kanade) video → {VIDEO_OUT}\n📈 Metrics CSV → {CSV_OUT}")


/var/folders/fy/j2skn70j52dfcbvjgdk6phr00000gn/T/ipykernel_69798/447498996.py:106: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  rgb = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
/var/folders/fy/j2skn70j52dfcbvjgdk6phr00000gn/T/ipykernel_69798/447498996.py:106: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  rgb = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
/var/folders/fy/j2skn70j52dfcbvjgdk6phr00000gn/T/ipykernel_69798/447498996.py:106: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  rgb = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
/var/folders/fy/j2skn70j52dfcbvjgdk6phr00000gn/T/ipykernel_69798/447498996.py:106: MatplotlibDeprecationWarning: The tostring_r

✅ DefMap (Lucas‑Kanade) video → /Users/macbook/Library/Mobile Documents/com~apple~CloudDocs/Studium/Spezialisierung/CodeExampleRayan/data/output/defmap_video_lk/defmap_lk_plasma.mp4
📈 Metrics CSV → /Users/macbook/Library/Mobile Documents/com~apple~CloudDocs/Studium/Spezialisierung/CodeExampleRayan/data/output/defmap_video_lk/defmap_lk_speed_metrics.csv


/var/folders/fy/j2skn70j52dfcbvjgdk6phr00000gn/T/ipykernel_69798/447498996.py:106: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  rgb = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
/var/folders/fy/j2skn70j52dfcbvjgdk6phr00000gn/T/ipykernel_69798/447498996.py:106: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  rgb = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
